In [11]:
from PIL import Image, ImageOps
import numpy as np
import time
from copy import deepcopy
from paddleocr import PaddleOCR
from difflib import SequenceMatcher
ocr_model = PaddleOCR(lang='en',use_angle_cls=True,show_log=False)

In [300]:
def rotator(data,angle,initializing_point):
    for i in data:
        points = i[0]
        i_xave = sum([j[0] for j in i[0]])/4.0
        i_yave = sum([j[1] for j in i[0]])/4.0
        rotated_center = rotate_about_point(points,origin = initializing_point,angle = angle)
        i[0] = rotate_about_point(rotated_center,origin=(i_xave,i_yave),angle = -angle)
    return data

In [474]:
def unrotate(image):
    initializers = ['Description'] 
    
    image = ImageOps.exif_transpose(image)
    img = np.asarray(image)
    data = ocr_model.ocr(img)[0]
    
    start_time = time.time()
    initializing_point = None
    headers = []
    for i in initializers:
        for j in data:
            if i.casefold() in j[1][0].casefold() or SequenceMatcher(None, i.casefold(), j[1][0].casefold()).ratio()>0.7:
                j_ymax = max([k[1] for k in j[0]])
                j_ymin = min([k[1] for k in j[0]])
                initializing_point = j
                
                for k in data:
                    k_ymax = max([n[1] for n in k[0]])
                    k_ymin = min([n[1] for n in k[0]])
                    
                    if not (k_ymax>j_ymax and k_ymin>j_ymax) and not (k_ymax<j_ymin and k_ymin<j_ymin) and not any([n in k[1][0] for n in '1234567890']):
                        headers.append(k)
        if len(headers)>0:
            break
    prev_header_len = len(headers)
    limit = 1
    prev_error = 0
    while(limit!=5):
        all_angles = []
        initializing_xave =  sum([i[0] for i in initializing_point[0]])/4.0
        initializing_yave =  sum([i[1] for i in initializing_point[0]])/4.0
        initializer_mid_point = (initializing_xave,initializing_yave)
        for i in headers:
            i_xave = sum([j[0] for j in i[0]])/4.0
            i_yave = sum([j[1] for j in i[0]])/4.0
            dist = ((i_xave-initializing_xave)**2 + (i_yave-initializing_yave)**2)**0.5
            i_xlen = max([j[0] for j in initializing_point[0]]) - min([j[0] for j in initializing_point[0]])
            if i!=initializing_point and dist<i_xlen*1.5:
                angle = np.arctan2(i_yave-initializing_yave,i_xave-initializing_xave)
                if angle!=0:
                    sign = int(angle/abs(angle))
                else:
                    sign = 1
                dist_to_180 = np.pi - abs(angle)
                dist_to_0 = abs(angle)
                if dist_to_0<dist_to_180:
                    all_angles.append((-1*sign,dist_to_0))
                else:
                    all_angles.append((sign,dist_to_180))
                    
        angle = sorted(all_angles,key=lambda x:x[1])[0]
        data = rotator(data,angle[0]*angle[1],initializer_mid_point)
        headers = header_checker(data,initializers)

        prev_header_len = len(headers)
        prev_error += angle[0]*angle[1]
        limit += 1
    image = image.rotate(-np.degrees(prev_error),center = initializer_mid_point)
    image.show()
    print("Time Taken: %s seconds" % (time.time() - start_time))
    return data

In [475]:
def header_checker(data,initializers):
    headers = []
    for i in initializers:
        for j in data:
            if i.casefold() in j[1][0].casefold() or SequenceMatcher(None, i.casefold(), j[1][0].casefold()).ratio()>0.7:
                j_ymax = max([k[1] for k in j[0]])
                j_ymin = min([k[1] for k in j[0]])

                for k in data:
                    k_ymax = max([n[1] for n in k[0]])
                    k_ymin = min([n[1] for n in k[0]])

                    if not (k_ymax>j_ymax and k_ymin>j_ymax) and not (k_ymax<j_ymin and k_ymin<j_ymin) and not any([n in k[1][0] for n in '1234567890']):
                        headers.append(k)
        if len(headers)>0:
            break
    return headers

In [476]:
def rotate_about_point(p, origin=(0, 0), angle=0):
    R = np.array([[np.cos(angle), -np.sin(angle)],
                  [np.sin(angle),  np.cos(angle)]])
    o = np.atleast_2d(origin)
    p = np.atleast_2d(p)
    return np.squeeze((R @ (p.T-o.T) + o.T).T).tolist()

In [481]:
img_path = "../data/rotational_test_sample.jpg"
# img_path = "../data/rotational_test_sample_2.jpg"
image = Image.open(img_path).convert("RGB")
output = unrotate(image)

Time Taken: 4.157412052154541 seconds


In [480]:
output

[[[[-15.387162298973522, 18.915186150966576],
   [288.61283770102654, 49.91518615096663],
   [285.6128377010265, 74.91518615096658],
   [-17.38716229897352, 44.91518615096663]],
  ('KIMFNG MINI SIPFRMARKFT', 0.8865070939064026)],
 [[[924.0355647467583, -65.95298753826012],
   [1206.0355647467584, -34.95298753826012],
   [1205.035564746758, -7.952987538260118],
   [921.035564746758, -37.95298753826012]],
  ('COMPANY REG NO:199102', 0.9702349305152893)],
 [[[1040.4393834587934, -76.64850357818909],
   [1204.4393834587934, -67.64850357818898],
   [1202.4393834587936, -45.64850357818909],
   [1038.4393834587931, -57.648503578189036]],
  ('OM2-0099779-4', 0.948551595211029)],
 [[[-13.393299733031544, 59.41895556545154],
   [67.60670026696852, 64.41895556545154],
   [66.60670026696846, 87.41895556545154],
   [-14.393299733031487, 81.41895556545157]],
  ('K103', 0.9747558832168579)],
 [[[470.55453173814595, 13.385547947183284],
   [602.5545317381459, 29.385547947183397],
   [599.5545317381459